<a href="https://colab.research.google.com/github/krajkumar6/deeplearning/blob/master/Image_Classification_MLP_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-Layer Perceptron, MNIST**

---

In this notebook, we will train an MLP to classify images from the MNIST database hand-written digit database.

The process will be broken down into the following steps:

1. Load and visualize the data
2. Define a neural network
3. Train the model
4. Evaluate the performance of our trained model on a test dataset!

Before we begin, we have to import the necessary libraries for working with data and PyTorch.

In [0]:
import torch
import numpy as np

# **Load and Visualize the Data**
Downloading may take a few moments, and you should see your progress as the data is loading. You may also choose to change the  batch_size if you want to load more data at a time.

This cell will create DataLoaders for each of our datasets.

In [0]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2


# convert data to torch.FloatTensor
transform = transforms.ToTensor()

train_data = datasets.MNIST(root='data',train = True, download = True, transform = transform)
test_data = datasets.MNIST(root='data',train=False, download = True, transform = transform)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

#prepare data samplers

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# Prepare data loaders

train_loader = torch.utils.data.DataLoader(train_data, batch_size= batch_size,sampler=train_sampler,  num_workers = num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size= batch_size, sampler=valid_sampler,num_workers = num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= batch_size, num_workers = num_workers)

## Visualize a Batch of Training Data
The first step in a classification task is to take a look at the data, make sure it is loaded in correctly, then make any initial observations about patterns in that data.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

## Define the Network Architecture
The architecture will be responsible for seeing as input a 784-dim Tensor of pixel values for each image, and producing a Tensor of length 10 (our number of classes) that indicates the class scores for an input image. This particular example uses two hidden layers and dropout to avoid overfitting

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  
  def __init__(self):
    super(Net,self).__init__()
    
    #number of hidden nodes in each layer is 512
    hidden_1 = 512
    hidden_2 = 512
    
    #linear layer (784 ---> hidden_1)
    self.fc1 = nn.Linear(28 * 28, hidden_1)
    
    #linear layer (hidden_1 ---> hidden_2)
    self.fc2 = nn.Linear(hidden_1,hidden_2)
    
    #linear layer (hidden_2 ---> 10)
    self.fc3 = nn.Linear(hidden_2, 10)
    
    #dropout layer
    self.dropout = nn.Dropout(0.2)
    
  def forward(self,x):
    
    #flatten the image
    x = x.view(-1,28*28)
    
    # add hidden layer, with relu activation function
    x = F.relu(self.fc1(x))
    
    # add dropout layer
    x = self.dropout(x)
    
    # add hidden layer, with relu activation function
    x = F.relu(self.fc2(x))
    
     # add dropout layer
    x = self.dropout(x)
    
    # add output layer
    x = self.fc3(x)
    
    return x
  
  
# initialize the NN
model = Net()
print(model)

# Specify Loss Function and Optimizer
It's recommended that you use cross-entropy loss for classification. If you look at the documentation (linked above), you can see that PyTorch's cross entropy function applies a softmax funtion to the output layer and then calculates the log loss.

In [0]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

# Train the Network
The steps for training/learning from a batch of data are described in the comments below:


1.  Clear the gradients of all optimized variables
2.  Forward pass: compute predicted outputs by passing inputs to the model
3. Calculate the loss
4. Backward pass: compute gradient of the loss with respect to model parameters
5. Perform a single optimization step (parameter update)
6. Update average training loss


The following loop trains for 50 epochs; take a look at how the values for the training loss decrease over time. We want it to decrease while also avoiding overfitting the training data.

In [0]:
n_epochs = 50

valid_loss_min = np.Inf

model.train() #prep the model for training

for epoch in range(n_epochs):
  
  train_loss = 0.0
  valid_loss = 0.0
  #train the model
  for data,target in train_loader:
    # clear the gradients of all optimized variables
    optimizer.zero_grad()
    
#     print(data.size(0))
    output = model(data)
    
#     print(output.shape)    
    loss = criterion(output, target)
    
#   print(loss.item())
    loss.backward()
    
    optimizer.step() #update parameters
    
    train_loss += loss.item()*data.size(0)
  
  
  #  Calculating Validation Loss
  model.eval() #set model in evaluation mode
  for data,target in valid_loader:
    output = model(data)
    loss = criterion(output,target)
    valid_loss += loss.item()*data.size(0)
  #   print training & validation statistics
  train_loss = train_loss/len(train_loader.dataset)
  valid_loss = valid_loss/len(valid_loader.dataset)
  
  print('Epoch: {} \tTraining Loss: {:.6f} \t Validation Loss: {:.6f}'.format(epoch+1,train_loss,valid_loss))
  
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased from {:.6f} to {:.6f}. Saving model..'.format(valid_loss_min,valid_loss))
    valid_loss_min = valid_loss
    torch.save(model.state_dict(),'model.pt')
    

# Test the Trained Network
Finally, we test our best model on previously unseen test data and evaluate it's performance. Testing on unseen data is a good way to check that our model generalizes well. It may also be useful to be granular in this analysis and take a look at how this model performs on each class as well as looking at its overall loss and accuracy.

In [0]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
# print(class_correct)
class_total = list(0. for i in range(10))

model.eval() #put the model in evaluation mode

for data, target in test_loader:
  output = model(data)
  loss = criterion(output, target) # calculate test loss
  test_loss += loss.item()*data.size(0)
  #   convert output probabilities to predicted class
  _,pred = torch.max(output,1)
  correct = np.squeeze(pred.eq(target.data.view_as(pred)))
  
  # calculate test accuracy for each object class
  for i in range(len(target)):
    label = target.data[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1
    
# calculate and print avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss:{:.6f}\n'.format(test_loss))

for i in range(10):
  if class_total[i] > 0:
#     print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (str(i), 100 * class_correct[i] / class_total[i],np.sum(class_correct[i]), np.sum(class_total[i])))
    print('Test Accuracy of {0}: {1:2.2f} ({2:5.2f}/{3:5.2f})'.format(str(i), 100 * class_correct[i] / class_total[i],class_correct[i],class_total[i]))
  else:
    print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),np.sum(class_correct), np.sum(class_total)))
 